In [ ]:
import os
import sys
import argparse
import pandas as pd
import time
from datetime import datetime
import matplotlib as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.optimizers import Adam, SGD

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics as metrics

from keras.models import load_model
from keras.models import model_from_json

import tensorflow as tf
from tensorflow.python.client import device_lib

import cv2
import json
from fastai_utils import *
from imgaug import augmenters as iaa
import imgaug as ia
from model_utils import *

start = time.time()

os.chdir('/data/data_backup_affine/Data_v2')

In [ ]:
bs = 64
nb_epoch = 5
train_data_path = "./final_train"
validation_data_path = "./final_validation"
test_path = "./final_test"
annotation_path = "./final_data_annotation.csv"
model_weights_path = "./models/model_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
img_channels = 3
img_rows = 224
img_cols = 224

In [ ]:
## Assign GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
model_split_1 = 'res4a_branch2a'
model_split_2 = 'fc_start'

## Defining model name
date = datetime.now().strftime("%m%d%Y")
model_name = 'resnet50_multioutput_noaug'
save_model_path = './models/iterations/' + date + '_' + model_name + '/'
if not os.path.exists(save_model_path):
    os.mkdir(save_model_path)
    
save_model_path = "./models/iterations/imgaug_1552532559/"

In [ ]:
## Reading Data Annotation
data = pd.read_csv(annotation_path, encoding = 'latin-1')

## Encoding the classes
def one_hot_categorical(x, classes):
    lb_enc = LabelEncoder().fit(np.array(classes).reshape(-1,1))
    label_encoded = lb_enc.transform(x)   
    enc = OneHotEncoder(sparse=False).fit(lb_enc.transform(np.array(classes).reshape(-1,1)).reshape(-1,1))
    enc.transform(label_encoded.reshape(-1,1))
    return enc.transform(label_encoded.reshape(-1,1)), lb_enc

## List of all classes of all columns
classes = {}
## List of one-hot encoded values for all columns
y_cols = []
## List of label encoders of all columns
lb = []
lb_dict = {}
## List of column names (as defined in the header of final_data_annotation.csv)
columns = ['Col' + str(i) for i in range(3, 8)]

for col in columns:
    classes[col] = list(set(data[col]))
    y_c, lb_c = one_hot_categorical(data[col], classes[col])
    y_cols.append(y_c)
    lb.append(lb_c)
    lb_dict[col] = dict(zip(lb_c.transform(lb_c.classes_).astype(str), (lb_c.classes_).astype(str)))

with open(save_model_path + 'label_encodings.json', "w") as json_file:
    json.dump(lb_dict, json_file)  
    
tot_num_classes = np.sum(len(c) for c in classes.values())

split_idx = [np.sum(y_cols[j].shape[1] for j in range(i)) for i in range(len(y_cols))][1:]

## Preparing dataframe that will be used as an input to flow_from_directory
targets = np.concatenate(y_cols, axis = 1)
image_files = pd.DataFrame(targets)
image_files['filename'] = data['filename']

In [ ]:
## Defining augmentation techniques
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        # iaa.Fliplr(0.5), # horizontally flip 50% of all images
        iaa.OneOf([
            sometimes(iaa.CropAndPad(percent=(-0.05, 0.1))), # zoom in
            sometimes(iaa.Affine(scale={"x": (0.6, 1.2), "y": (0.6, 1.2)})) # zoom out
        ]), 
        # execute 1 to 2 of the following augmenters per image
        sometimes(iaa.Affine(translate_px={"x": (0, 25), "y": (0, 25)}, # horizontal/vertical shift
                            rotate = (-25, 25),
                            shear = (-15, 15))), 
        iaa.SomeOf((1, 4),
                   [iaa.OneOf([
                       iaa.PerspectiveTransform(scale=(0.01, 0.07)),
                       iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)),
                       iaa.PiecewiseAffine((0.0, 0.01)), # local distortions
                       iaa.GaussianBlur(sigma=(0, 0.7))
                   ]),
                    sometimes(iaa.Dropout((0.01, 0.02), per_channel=0.5)),
                    iaa.AdditiveGaussianNoise(loc=32, scale=0.01*255), # white noise
                    iaa.Add((-20, 50)), # brightness
                    iaa.OneOf([
                        iaa.LinearContrast(alpha=(0.5,1.2), per_channel = True),
                        iaa.ContrastNormalization((0.5, 1.0))
                    ])
            ], random_order=True)
    ], random_order=True)

In [ ]:
## Defining Data Generator
def split_outputs(generator, idx):
    while True:
        data = next(generator)
        x = data[0]
        y = np.split(data[1], idx, axis=1)
        yield x, y

def read_data(img_data_gen, base_dir, in_df, idx, path_col = 'filename',
                        y_col = 'targets', batch_size = 8, n_classes = 38, rows = 224, cols = 224):
    df_gen = img_data_gen.flow_from_dataframe(in_df, base_dir,
                                              x_col = 'filename', y_col= list(range(n_classes)),
                                              has_ext = False, target_size = (rows, cols),
                                              color_mode = 'rgb', class_mode = 'other',
                                              batch_size = batch_size)
    return split_outputs(df_gen, idx)

In [ ]:
## Creating image generators for train and validation
img_train_gen = ImageDataGenerator(rescale=1/255, preprocessing_function = seq.augment_image)
img_val_gen = ImageDataGenerator(rescale=1/255)

## Splitting annotated images into training and validation
## train
train_ids = list(set(data.loc[data['split'] == 'train', 'filename']))
train_df = image_files.loc[image_files['filename'].isin(train_ids),:].drop_duplicates().reset_index(drop = True)
print(str(len(train_ids)) + ' Training Images')
train_gen = read_data(img_train_gen, train_data_path, train_df, idx = split_idx,
                                batch_size = bs, n_classes = tot_num_classes, rows = img_rows, cols = img_cols)
## validation
val_ids = list(set(data.loc[data['split'] == 'val', 'filename']))
val_df = image_files.loc[image_files['filename'].isin(val_ids),:].drop_duplicates().reset_index(drop = True)
print(str(len(val_ids)) + ' Validation Images')
val_gen = read_data(img_val_gen,validation_data_path, val_df,idx = split_idx,
                              batch_size = bs,  n_classes = tot_num_classes, rows = img_rows, cols = img_cols)

In [ ]:
def build_model(base_model, img_channels, img_w, img_h, classes_list):    
    """Builds and returns a learning model.
    img_channels: the number of channels in the input images (1 for grayscale,
        or 3 for RGB).
    img_w: the width (in pixels) of the input images.
    img_h: the height of the input images.
    classes_list: the number of classes that the data will have (for each code) - this dictates
        the number of values produced in the output layer.
    Returns:
    A deep neural network model.
    """
    
    x = base_model.output
    x = Conv2D(filters = 512,kernel_size = 3,strides=(1,1), name = 'fc_start')(x)
    x = BatchNormalization()(x)

    # Col 3
    x3 = GlobalAveragePooling2D()(x)
    x3 = Dense(1024, activation='relu')(x3)
    x3 = Dropout(0.2)(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dense(512, activation='relu')(x3)
    x3 = Dropout(0.2)(x3)
    out3 = Dense(len(classes_list['Col3']), activation='softmax', name = 'col3')(x3)

    # Col 4
    x4 = Conv2D(filters = 256,kernel_size = 3,strides=(1,1))(x)
    x4 = GlobalAveragePooling2D()(x4)
    x4 = Dense(1024, activation='relu')(x4)
    x4 = Dropout(0.5)(x4)
    x4 = BatchNormalization()(x4)
    x4 = Dense(512, activation='relu')(x4)
    x4 = Dropout(0.5)(x4)
    x4 = BatchNormalization()(x4)
    x4 = Dense(512, activation='relu')(x4)
    x4 = Dropout(0.5)(x4)
    out4 = Dense(len(classes_list['Col4']), activation='softmax', name = 'col4')(x4)

    # Col 5
    x5 = GlobalAveragePooling2D()(x)
    x5 = Dense(1024, activation='relu')(x5)
    x5 = Dropout(0.2)(x5)
    x5 = BatchNormalization()(x5)
    x5 = Dense(512, activation='relu')(x5)
    x5 = Dropout(0.2)(x5)
    out5 = Dense(len(classes_list['Col5']), activation='softmax', name = 'col5')(x5)

    # Col 6
    x6 = GlobalAveragePooling2D()(x)
    x6 = Dense(1024, activation='relu')(x6)
    x6 = Dropout(0.2)(x6)
    x6 = BatchNormalization()(x6)
    x6 = Dense(512, activation='relu')(x6)
    x6 = Dropout(0.2)(x6)
    out6 = Dense(len(classes_list['Col6']), activation='softmax', name = 'col6')(x6)

    # Col 7
    x7 = Conv2D(filters = 256,kernel_size = 3,strides=(1,1))(x)
    x7 = GlobalAveragePooling2D()(x7)
    x7 = Dense(1024, activation='relu')(x7)
    x7 = Dropout(0.5)(x7)
    x7 = BatchNormalization()(x7)
    x7 = Dense(512, activation='relu')(x7)
    x7 = Dropout(0.5)(x7)
    out7 = Dense(len(classes_list['Col7']), activation='softmax', name = 'col7')(x7)

    return [out3, out4, out5, out6, out7]

In [ ]:
# Defining f1 Score - for model evaluation
def f1(y_true, y_pred):

    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)

    TP = tf.count_nonzero(y_pred * y_true, axis=0, dtype = tf.float64)
    FP = tf.count_nonzero(y_pred * (y_true - 1), axis=0, dtype = tf.float64)
    FN = tf.count_nonzero((y_pred - 1) * y_true, axis=0, dtype = tf.float64)

    add_dummy = lambda x: x + K.epsilon()
    precision = TP /  tf.map_fn(add_dummy, (TP + FP))
    recall = TP / tf.map_fn(add_dummy, (TP + FN))
    f1 = 2 * precision * recall / tf.map_fn(add_dummy, (precision + recall))

    weights = tf.reduce_sum(y_true, axis=0)
    weights /= tf.reduce_sum(weights)
    return tf.reduce_sum(f1 * weights)

In [ ]:
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

## Defining the input image size
SHAPE = (img_rows, img_cols, img_channels)

base_model = ResNet50(include_top=False, weights=None, input_shape = SHAPE)
base_model.load_weights(model_weights_path)

print('freezing entire base model ...\n')
# Freezing entire base model
for layer in base_model.layers[:]:
    layer.trainable = False

model_output = build_model(base_model, img_channels, img_rows, img_cols, classes)
model = Model(inputs = base_model.input, outputs = model_output)

model_checkpoint = ModelCheckpoint(save_model_path + model_name +'_checkpoint.h5',
                                   monitor='val_loss', 
                                   mode = 'auto', save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor='val_loss', mode = 'auto',patience = 5, verbose=2)
# For cyclical Learning Rate
sched = LR_Cycle(iterations = np.ceil(len(train_ids)/bs),
                 cycle_mult = 2)

# cbks = [model_checkpoint,early_stopping,reduce_lr]
cbks1 = [model_checkpoint,early_stopping]
cbks2 = [model_checkpoint,early_stopping, sched]

opt = SGD(lr = 5 * 1e-2, momentum = 0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[f1,"accuracy"])

print('running 2 epochs with non-trainable base layers ...\n')
history = model.fit_generator(train_gen,
                              steps_per_epoch = np.ceil(len(train_ids)/bs),
                              epochs = 2,
                              validation_data = val_gen,
                              validation_steps = np.ceil(len(val_ids)/bs),
                              use_multiprocessing = True,
                              callbacks = cbks1)



print('unfreezing all layers ...\n')
for layer in model.layers:
    layer.trainable = True


# For Differential Learning Rate
split_layer_1 = [layer for layer in model.layers if layer.name == model_split_1][0]
split_layer_2 = [layer for layer in model.layers if layer.name == model_split_2][0]

opt = SGD_dlr(split_l1 = split_layer_1,
              split_l2 = split_layer_2,
              lr = [1e-10, 1e-6, 1e-3],
              momentum = 0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=[f1,"accuracy"])

history = model.fit_generator(train_gen,
                              steps_per_epoch = np.ceil(len(train_ids)/bs),
                              epochs = nb_epoch - 2,
                              validation_data = val_gen,
                              validation_steps = np.ceil(len(val_ids)/bs),
                              use_multiprocessing = True,
                              callbacks = cbks2)


In [ ]:
## Save model and weights
model_json = model.to_json()
with open(save_model_path + model_name + '_model.json', "w", encoding = 'utf-8') as json_file:
    json_file.write(model_json)
model.save_weights(save_model_path + model_name + '_weights.h5')

#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours") 

In [ ]:
## Reading the true annotations
data = pd.read_csv(annotation_path, encoding = 'latin-1')

image_names = [i for i in os.listdir(test_path) if 'Thumbs' not in i]

data['filename'] = data['filename'] + '.jpg'
test_df = data.loc[data['filename'].isin(image_names)].drop_duplicates().reset_index(drop = True)
print(data.shape[0], 'test images found ...')

model_path = save_model_path
## Extract the encodings
lb_dict = json.loads(open(model_path + 'label_encodings.json').read())
lb = []
## List of all classes of all columns
classes = {}
## List of column names (as defined in the header of final_data_annotation.csv)
columns = ['Col' + str(i) for i in range(3, 8)]
for col in columns:
    classes[col] = list(lb_dict[col].values())
    lb.append(lb_dict[col])
tot_num_classes = np.sum(len(c) for c in classes.values())

## Load model and weights
print('loading the model ...\n')
json_file = open(model_path + 'resnet50_multioutput_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json, 
                        custom_objects={'f1':f1})
model.load_weights(model_path + 'resnet50_multioutput_checkpoint.h5')

## Defining augmentation techniques for Test Time Augmentation
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential(
    [
        # apply the following augmenters to most images
        iaa.OneOf([
            sometimes(iaa.CropAndPad(percent=(-0.05, 0.1))), # zoom in
            sometimes(iaa.Affine(scale={"x": (0.6, 1.2), "y": (0.6, 1.2)})) # zoom out
        ]), 
        # execute 1 to 2 of the following augmenters per image
        sometimes(iaa.Affine(translate_px={"x": (0, 10), "y": (0, 10)},
                            rotate = (-25, 25)
                            )
                 ), # horizontal/vertical shift
        iaa.SomeOf((1, 4),
                   [iaa.OneOf([
                       iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.5)),
                       iaa.GaussianBlur(sigma=(0, 0.7))
                   ]),
                    sometimes(iaa.Dropout((0.01, 0.02), per_channel=0.5)),
                    iaa.AdditiveGaussianNoise(loc=32, scale=0.01*255), # white noise
                    iaa.Add((-20, 50)), # brightness
                    iaa.OneOf([
                        iaa.LinearContrast(alpha=(0.5,1.2), per_channel = True),
                        iaa.ContrastNormalization((0.5, 1.0))
                    ])
            ], random_order=True)
    ], random_order=True)

## Get actual code
y_true = [np.array(test_df[col].astype(str)) for col in columns]

## Get predicted code
pred = []
for ind in range(test_df.shape[0]):

    image_n = test_df.loc[ind, 'filename']
    ## Read Image
    img = cv2.cvtColor(cv2.imread(test_path + '/' + image_n), cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_rows, img_cols))

    images_l = []
    ## Original Image
    images_l.append(img/255)
    ## Augmented Images
    for aug in range(15):
        images_l.append(seq.augment_image(img)/255)

    img_predictions = []
    for image in images_l:
        img_predictions.append(model.predict(image.reshape((-1,img_rows, img_cols, 3))))
    predictions = []
    label_pred = []
    for c in range(len(classes)):
        class_pred = np.array(([(img_predictions[j][c]).reshape(-1) for j in range(len(img_predictions))]))
        predictions.append(np.mean(class_pred, axis = 0))
        label_pred.append(lb[c][str(np.argmax(predictions[c]))])
    pred.append(label_pred)

y_pred = []
for c in range(len(classes)):
    y_pred.append(np.array([i[c] for i in pred]))

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=False):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(5, 5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
def cm_model_report(writer, folder, true, pred, cols, lb):

    total_samples = len(true[0])
    start = 0
    m = []
    print(str(['Column', 'Precision', 'Recall', 'Accuracy']))
    for i in range(len(cols)):

        class_true = true[i]
        class_pred = pred[i]

        # Accuracy Metrics
        prf = np.round(metrics.precision_recall_fscore_support(class_true, class_pred, average='weighted')[:-1],4)
        acc = np.round(metrics.accuracy_score(class_true, class_pred),4)
        metric_list = [cols[i]]
        metric_list.extend(prf)
        metric_list.extend([acc])
        m.append(metric_list)
        print(str(metric_list))

        # Confusion Matrix
        report = metrics.confusion_matrix(class_true, class_pred, labels = list(lb[i].values()))
        plot_confusion_matrix(report, list(lb[i].values()), title = cols[i])
        df = pd.DataFrame(report, columns = list(lb[i].values()), index = list(lb[i].values()))
        df.to_excel(writer,sheet_name='CM_' + folder ,startrow = start , startcol=0)   

        # Class Scores
        df_class_score = pd.DataFrame(index = lb[i].values())
        df_class_score['Precision'] = metrics.precision_score(class_true, class_pred,
                                                              average = None, labels = list(lb[i].values()))
        df_class_score['Recall'] = metrics.recall_score(class_true, class_pred,
                                                        average = None, labels = list(lb[i].values()))
        df_class_score['F1'] = metrics.f1_score(class_true, class_pred,
                                                average = None, labels = list(lb[i].values()))
        df_class_score.to_excel(writer, sheet_name = folder + '_score', startrow = start, startcol = 0)

        start = start + len(list(lb[i].values())) + 2

    metrics_df = pd.DataFrame(m, columns = ['Column','Precision','Recall','F1','Accuracy'])
    metrics_df.to_excel(writer, sheet_name=folder, index = False)
    pd.DataFrame({'Samples':[total_samples]}).to_excel(writer, sheet_name=folder, startrow = 8, index = False)
    print()

    return writer

In [ ]:
writer = pd.ExcelWriter(model_path + 'overall_report.xlsx', engine='xlsxwriter')
writer = cm_model_report(writer, 'test', y_true, y_pred, columns, lb) 
writer.save()

In [ ]:
actuals = pd.DataFrame(y_true).T
actuals.columns = ['Actual_' + col for col in columns]
predicted = pd.DataFrame(y_pred).T
predicted.columns = ['Predicted_' + col for col in columns]
df = pd.concat([test_df['filename'], actuals, predicted], axis = 1)
df.to_csv(model_path + 'actuals_vs_predicted.csv', index = False)